In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
train = datasets.MNIST("", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

/usr/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
trainset = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)

In [4]:
total = 0

counter_dict = {x:0 for x in range(10)}

for data in trainset:
    Xs, ys = data
    for y in ys:
        counter_dict[int(y)] += 1
        total +=1
for i in counter_dict:
    counter_dict[i] = round(counter_dict[i]/total*100, 2)

print(counter_dict)

{0: 9.87, 1: 11.24, 2: 9.93, 3: 10.22, 4: 9.74, 5: 9.04, 6: 9.86, 7: 10.44, 8: 9.75, 9: 9.92}


In [5]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x

net = Net()

optimizer = optim.Adam(net.parameters(), lr=1e-2)

In [6]:
EPOCHS = 3

for epoch in range(EPOCHS):
    for data in trainset:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1, 28*28))
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print(epoch, loss)

0 tensor(0.3694, grad_fn=<NllLossBackward>)
1 tensor(0.2433, grad_fn=<NllLossBackward>)
2 tensor(0.5426, grad_fn=<NllLossBackward>)


In [7]:
train_correct = 0
train_total = 0

with torch.no_grad():
    for data in trainset:
        X, y = data
        output = net(X.view(-1, 28*28))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                train_correct +=1
            train_total += 1

print("Accuracy:", round(train_correct/train_total*100, 2))

Accuracy: 96.09


In [8]:
test_correct = 0
test_total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1, 28*28))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                test_correct +=1
            test_total += 1

print("Accuracy:", round(test_correct/test_total*100, 2))

Accuracy: 95.27
